In [2]:
!pip install pykeops[full] > log.log


# Nystrom implementation

In [ ]:
from sklearn import datasets, svm
import time
import numpy as np
import torch
from matplotlib import pyplot as plt
from pykeops.torch import LazyTensor
from scipy.linalg import svd

In [ ]:
def pairwise_kernels_keops(x,y=None, sigma=0.1):
  """A kernel matrix K such that K_{i, j} is the kernel between the ith and jth 
  vectors of the given matrix X"""
  #TODO: check pairwise arrays etc for Value Error
  if y==None:
    y=x
  x_i = LazyTensor(x[:, None, :])  # (M, 1, 1)
  y_j = LazyTensor(y[None, :, :])  # (1, N, 1)
  print(123)
  D_ij = ((x_i - y_j) ** 2).sum(-1)  # (M, N) symbolic matrix of squared distances
  print(type(D_ij))
  return (-D_ij / (2 * sigma ** 2)).exp()  # (M, N) symbolic Gaussian kernel matrix
  


def pairwise_kernels(x, y=None, sigma =0.1):
  if y==None:
    y=x
  x_i = x[:, None, :]  # (M, 1, 1)
  y_j = y[None, :, :] # (1, N, 1)
  D_ij = ((x_i-y_j) ** 2).sum(-1)  # (M, N) symbolic matrix of squared distances
  print(type(D_ij))
  return (-D_ij / (2 * sigma ** 2)).exp()  # (M, N) symbolic Gaussian kernel matrix
    



In [ ]:
class Nystrom:
    '''Class to implement Nystrom'''

  def __init__(self, n_components=100):
    self.n_components = n_components
    #self.normalization_=None

  def fit(self, X, y):
    #number of samples. must be greater than n_components
    n_samples = X.shape[0]

    #define basis
    n_components = min(n_samples, self.n_components) #should be = n_samples
    inds = torch.randperm(n_samples) #rnd.permutation(n_samples)
    basis_inds = inds[:n_components]
    basis = X[basis_inds]
    # get rbf kernel matrix
    basis_kernel = pairwise_kernels(basis, y)

    # get svd
    U, S, V = torch.svd(basis_kernel) 
    u = U * 1. / torch.sqrt(S)
    self.normalization_ = torch.dot(u[:,0], V[:,0]) #????
    self.components_ = basis
    self.component_indices_ = inds
   
    return basis_kernel


